<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing II</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=463">MICRO-512</a>) taught by Dr. D. Sage, Dr. M. Liebling, Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL <mark>2024</mark>.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:pol.delaguilapla@epfl.ch">Pol del Aguila Pla</a>, 
    <a href="mailto:kay.lachler@epfl.ch">Kay Lächler</a>,
    <a href="mailto:alejandro.nogueronaramburu@epfl.ch">Alejandro Noguerón Arámburu</a>,
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>, and
    <a href="mailto:kamil.seghrouchni@epfl.ch">Kamil Seghrouchni</a>.
     
</p>
<hr style="clear:both">
<h1>Lab 5.1: Geometric transformation - Implementations</h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: <mark>Thursday 14th of March, 2024</mark></p>
    <p style="margin:4px;"><b>Submission</b>: <mark><span style="color:red">Monday 25th of March 2024</span></mark> (before 11:59PM)
    <p style="margin:4px;"><b>Help sessions</b>: <mark>Thursday 21st of March 2024</mark> in CM 1 2</p>
    <p style="margin:4px;"><b>Related lectures</b>: Chapter 7</p>
</div>

### Student Name: 
### SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [ ]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

## <a name="imports_"></a> Imports
In the next cell we import standard Python libraries that we will use throughout the lab, as well as the following libraries that are required for the exercises:

* [`matplotlib.pyplot`](https://matplotlib.org/3.2.2/api/_as_gen/matplotlib.pyplot.html), to display images,
* [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/), to make the image display interactive,
* [`numpy`](https://numpy.org/doc/stable/reference/index.html), for mathematical operations on arrays,
* [`openCV` (cv2)](https://docs.opencv.org/2.4/index.html), for image processing tasks,
* [`scipy`(scipy)](https://www.scipy.org), also for image processing tasks.

We will then load the `ImageViewer()` class as `viewer`. You can find the documentation of the class [here](https://github.com/Biomedical-Imaging-Group/interactive-kit/wiki/Image-Viewer).

Finally, we load the images you will use in the exercise to test your functions. 

In [ ]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import ipywidgets as widgets
from scipy import ndimage
from interactive_kit import imviewer as viewer 

# Load images to be used in this exercise with data type float 64
eiffel = cv.imread('images/eiffel.png',cv.IMREAD_UNCHANGED).astype(np.float64)

We also import the JavaScript `ImageAccess` class as `Image`. You can find the documentation of the class [here](https://biomedical-imaging-group.github.io/image-access/).

In [ ]:
%use javascript
%get eiffel
// import image-access as Image
var Image = require('image-access')

# Geometric transformation - Implementations (15 points)

In this laboratory you will learn how to implement geometric transfomations of image data, as well as several interpolators, in a low-level language like JavaScript. You will then use your implemenations for image processing applications. We will focus on geometric transformations of two-dimensional grayscale images, but keep in mind that the same operations can be adapted to color images by treating each color channel as an independent grayscale image.

## <a id="ToC_1_GT_Implementations"></a>Table of contents
1. [Understanding geometry](#1.-Understanding-geometry)
    1. [Rotation of the coordinate frame](#1.A.-Rotation-of-the-coordinate-frame-(1-point)) **(1 point)** 
    2. [Implementing the geometric transformation](#1.B.-Implementing-the-geometric-transformation-(4-points)) **(4 points)** 
    3. [Visualizing transformations](#1.C.-Visualizing-transformations)
    4. [Geometric transformations in Python](#1.D.-Geometric-transformations-in-Python-(1-point)) **(1 point)**
2. [B-spline interpolation scheme and linear interpolation](#2.-B-spline-interpolation-scheme-and-linear-interpolation)
    1. [Understanding the method](#2.A.-Understanding-the-method-(1-point)) **(1 point)** 
    2. [Linear interpolation](#2.B.-Linear-interpolation-(3-points)) **(3 points)** 
3. [Cubic interpolation](#3.-Cubic-interpolation) 
    1. [Prefilter](#3.A.-Prefilter-(3-points)) **(3 points)** 
    2. [Cubic Interpolation](#3.B.-Cubic-interpolation-(2-points)) **(2 points)** 
    3. [Visual comparison between the three interpolators](#3.C.-Visual-comparison-between-the-three-interpolators) 


<div class=" alert alert-danger">
    
<b>Important:</b> Each cell that contains code begins with <code>%use sos</code> or <code>%use javascript</code>. This indicates if the code in this specific cell should be written in Python or JavaScript. Do not change or remove any lines of code that begin with an <code>%</code>. They need to be on the first line of each cell!
    
</div>

### Visualize images
Use the next cell to get familiar with the image you are going to be using.

In [ ]:
%use sos
# Display images

plt.close('all')
imgs_viewer = viewer(eiffel, widgets=True, hist=True)

# 1. Understanding geometry 
[Back to table of contents](#ToC_1_GT_Implementations)

In this section we will study the basics of geometric transformations. We will start by understanding how one moves a coordinate frame using a coordinate transformation matrix. Then, we will look at how one obtains the transformed image by iterating through the pixels in the new image and retrieving the value at their corresponding positions in the original image. As you probably remember from the course, the main challenge of geometric transformations is that you have to correctly interpolate the pixel values at new locations. In this section, we will use [*nearest neighbor*](https://en.wikipedia.org/wiki/Nearest-neighbor_interpolation) interpolation. 

After you learn how to apply a geometric transformation in a pixel-wise fashion, we will show you how to do it with *SciPy* in just a couple of lines, and we will make a direct comparison of your result to *SciPy*'s. 

## 1.A. Rotation of the coordinate frame (1 point)
[Back to table of contents](#ToC_1_GT_Implementations)

We parametrize a geometric transformation on a two-dimensional image by 1) the angle of rotation of the coordinate frame $\alpha$ (by convention, a positive angle is defined in the counter-clockwise direction), 2) the scaling factor $\rho>0$ (where $\rho > 1$ increases the size of the image, while $\rho < 1$ decreases it), and 3) the center of the transformation $\mathbf{c}=(c_1, c_2)$. Thus, the pixel at position $\mathbf{u}=(u_1, u_2)$ in the input image gets mapped to the position $\mathbf{v}=(v_1, v_2)$ in the output image according to

$$ (\mathbf{v}-\mathbf{c}) = \mathbf{A}(\mathbf{u} - \mathbf{c}) \Rightarrow \mathbf{v} = \mathbf{A}(\mathbf{u} - \mathbf{c}) + \mathbf{c}\,,$$

where $\mathbf{A}$ contains the effect of both the rotation of the coordinate frame by $\alpha$ and the scaling factor $\rho$. 

<div class = 'alert alert-warning'>

<b>Technical note:</b> The convention in image processing is to <b>rotate the coordinate frame</b>, as opposed to rotating a point while keeping the same coordinate frame. In fact, the two operations are the inverse of each other (rotating the coordinate frame by $\alpha$ will look like rotating the point by $-\alpha$). Explore the image below and make sure you understand the concept before continuing.
</div>

<table><tr>
<td> 
  <p align="center" style="padding: 10px">
    <img alt="Forwarding" src="images/coord_frame_rotation.png" width="500">
    <br>
    <em style="color: grey">Visual explanation of coordinate frame rotation (from <a href="https://datagenetics.com/blog/august32013/index.html">here</a>)</em>.
  </p> 
</td>
</tr></table>

### Multiple Choice Question

To start the lab and **for 1 point**, answer the following question. 

 * Among the choices below, which $2 \times 2$ matrix $A$ performs a rotation **of the coordinate frame** of angle $\alpha$ and a scaling by $\rho$? 
    1. $ A = \rho \left[\begin{array}{ccc} \cos(\alpha) & \sin(\alpha)\\ -\sin(\alpha) & \cos(\alpha) \end{array}\right]$,
    2. $ A = \rho \left[\begin{array}{ccc} \cos(\alpha) & -\sin(\alpha)\\ \sin(\alpha) & \cos(\alpha) \end{array}\right]$,
    3. $A = \frac{1}{\rho} \left[\begin{array}{ccc} \cos(\alpha) & \sin(\alpha)\\ -\cos(\alpha) & \sin(\alpha) \end{array}\right]$, or
    4. $A = \frac{1}{\rho}\left[\begin{array}{ccc} \cos(\alpha) & -\sin(\alpha)\\ \sin(\alpha) & \cos(\alpha) \end{array}\right]$.
    
<div class=" alert alert-info">

**Note:** To answer, change the variable `answer` in the cell below to the number corresponding to your choice. Then run the cell below to check that your answer is valid. 
</div>

In [ ]:
%use sos
# Modify the variable answer
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in [1, 2, 3, 4]:
    print('WARNING!\nValid answers are 1, 2, 3 and 4.')

## 1.B. Implementing the geometric transformation (4 points)

[Back to table of contents](#ToC_1_GT_Implementations)

When computing the geometric transformation of an image, one iterates over the pixels of the output image and retrieves the corresponding values from the input image. To know which pixel from the input image corresponds to each pixel of the output image, we need to solve the equation for $\mathbf{u}$ instead of $\mathbf{v}$, obtaining

$$\mathbf{u} = A^{-1}(\mathbf{v}-\mathbf{c}) + \mathbf{c}\,.$$

In the next cell, **for 2 points**, complete the JavaScript function `computeMatrix()` which returns the matrix $A^{-1}$ as a $2\times 2$ `Image` object. 

`computeMatrix(angleDegrees, scaling)` takes as input parameters
* `angleDegrees`: the angle of rotation of the coordinate frame $\alpha$ (**in degrees**),
* `scaling`: the scaling factor $\rho$,

and returns
* `Ainv`: an `Image` object containing $A^{-1}$, the inverse of the $2 \times 2$ transformation matrix $A$.

<div class = 'alert alert-info'>

<b>Hint:</b> The JavaScript functions <code>Math.sin</code> and <code>Math.cos</code> take as input an angle in <b>radians</b>. You should use of <code>Math.PI</code> when needed, which represents $\pi$ in JavaScript.
    
</div>

<div class = 'alert alert-info'>

<b>Hint:</b> Make sure you don't get confused by the order of the indices in <code>img.setPixel()</code> with respect to the usual ordering in matrices!
    
</div>

In [ ]:
%use javascript
// Function that return the inverse of the transformation matrix A
function computeMatrix(angleDegrees, scaling){
    // Initialize 2x2 image Ainv filled with zeros
    var Ainv = new Image(2,2);
    
    // YOUR CODE HERE
    
    return Ainv; 
}

In the next cell, we will run a sanity check for your function `computeMatrix`. We will apply your function on several angles and scale values. If the sanity check fails, try to understand the test and see where the error might come from.

<div class = 'alert alert-danger'>

<b>Remember:</b> As for all sanity checks, this is <b>NOT</b> a definitive test, and the fact that the cell runs does not guarantee that you will get the points. 
</div>    

In [ ]:
%use javascript
// declare solutions for 0° and a scaling of one, and for 90° and a scaling of 2.
var solution_test1 = new Image([[1, 0], [0, 1]]);
var solution_test2 = new Image([[0, -0.5], [0.5, 0]]);

// the transformationn with 0° angle and unit scale should produce an identity matrix
console.log('The inverse matrix for 0° rotation with unity scaling is the identity matrix as below.\nCorrect solution:\n' + solution_test1.visualize(2) )
console.log('Your result:\n' + computeMatrix(0, 1).visualize(2));
var check_error = false;
// compare your solution with the reference transformation
if(solution_test1.imageCompare(computeMatrix(0, 1)) == false){
    console.log('WARNING!\nYour computeMatrix() function is not yet correct for the first test case.');
    check_error = true;
}
// the transformation with 90° angle and scaling of 2 should be an anti-diagonal matrix with values of +/-0.5
console.log('The inverse matrix for 9O° rotation with a scaling of 2 is anti-diagonal and contains the values +/-0.5 as below.\nCorrect solution:\n' + solution_test2.visualize(2))
console.log('Your result:\n' + computeMatrix(90, 2).visualize(2));
//Compare your solution with the reference transformation
if(solution_test2.imageCompare(computeMatrix(90, 2)) == false){
    console.log('WARNING!\nYour computeMatrix() function is not yet correct for the second test case.');
    check_error = true;
}
if(!check_error){
    console.log('Nice, the computeMatrix() function is correct for the two test cases. Make sure to double check everything, however!');
}

For some transformations (for example a rotation by $180^\circ$), each pixel in the output image will correspond to an exact pixel position in the input image. However, most of the time this is not the case and the pixel value of the output image will correspond to a position in the input image that is between two pixels. In this case, we need to interpolate the corresponding value from the pixel values that are around that point. In this first exercise we provide you with a nearest neighbor interpolator. Later on, you will implement more sophisticated methods to interpolate the values. 

Run the next cell to declare the function `nn_interpolator()` which performs the nearest neighbor interpolation. Conveniently, when one has defined a grid in $\mathbb{R}^2$, finding the closest grid-point to any given location is as easy as rounding off each coordinate to the closest integer.

In [ ]:
%use javascript
//  Nearest neighbor interpolator 
function nn_interpolator(img, x,  y){
    return img.getPixel(Math.round(x), Math.round(y), padding='zero');
}

In the next cell, **for 2 points**, complete the function `transform()`, which will perform the complete geometric transformation of the image, taking as parameters
* `img`: the original image,
* `angle`: the rotation angle in degrees,
* `scaling`: the scaling factor,
* `c1` and `c2`: the $x$- and $y$-coordinates, respectively, of the center of rotation,
* `interpolator`: a string, which for now we will call `'NN'`, to refer to nearest neighbor interpolation,

and returning
* `out`: the transformed image.

In particular, you will need to complete the section of the code that computes the location in the original image, $\mathbf{u}$, from the inverse transformation matrix $\mathbf{A^{-1}}$, the center of the transformation $\mathbf{c}$, and the location in the output image $\mathbf{v}$, using the formula we specified at the [beginning of this section](#1.B.-Implementing-the-geometric-transformation-(4-points)).

In [ ]:
%use javascript
// Function that rotates and scales an image around a given center point
function transform(img, angle, scaling, c1, c2, interpolator){
    // Preallocate space for output image
    var out = new Image(img.shape());
    
    // Coefficients for each pixel in the input image (only after cubic spline interpolation)
    var coef = (interpolator == 'CUBIC') ? cubicSplineCoefficients(img) : null; 
    
    // Get the inverse of the rotation matrix A
    var Ainv = computeMatrix(angle, scaling);
    
    // Iterate over the positions in the output image v
    for(var v1 = 0; v1 < img.nx; v1++){
        for(var v2 = 0; v2 < img.ny; v2++){
            
            // Compute the corresponding position in the input image, u = (u1, u2) from
            // the output pixel position v = (v1, v2), Ainv, and the center c = (c1, c2).
            var u1 = 0;
            var u2 = 0;
            
            // YOUR CODE HERE
            
            // Check that the input pixel location is inside the image, otherwise leave the pixel at 0 (default value in out)
            if(u1 >= 0 && u2 >= 0 && u1 <= img.nx-1 && u2 <= img.ny-1){
                var value = 0; 
                // Select the interpolator
                switch(interpolator){
                    case "NN": 
                        value = nn_interpolator(img, u1, u2);
                        break;
                    case "LINEAR":
                        value = linear_interpolator(img, u1, u2);
                        break ; 
                    case "CUBIC": 
                        value = cubic_interpolator(coef, u1, u2);
                        break ;
                }
                // set pixel (v1,v2) of the output image to the interpolated value
                out.setPixel(v1, v2, value);
            }else{
                continue;
            }
        }
    }
    return out; 
}

Let's perform a quick sanity check. We will apply your function on a small $5\times 5$ cross image, then visualize and compare transformations by $0^\circ$ and scaling factor $2$, and $40^\circ$ and scaling factor $1.01$, respectively, to see the effect of the nearest neighbor interpolation. Both transformations will be centered at the middle of the image, i.e., $\mathbf{c}=[2,2]^\mathrm{T}$. If your cell raises an error, explore the test in detail to see where it might come from.

In [ ]:
%use javascript
// Declare the cross image used for testing and visualize
var test_cross_image  = new Image ([[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0]]);
console.log('Cross image:\n' + test_cross_image .visualize(4) + '\n---------------\n')
// Test for (angle, scale) = (0, 2) using NN interpolation 
var solution_test_0 = new Image([[0, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [0, 1, 1, 0, 0], [0, 1, 1, 0, 0]]);
var result_test_0 = transform(test_cross_image, 0, 2, 2, 2, 'NN');
console.log('Correct transformation with NN interpolation, (angle, scale) = (0, 2):\n' + solution_test_0.visualize(2))
console.log('Your result:\n' + result_test_0.visualize(2));
if(solution_test_0.imageCompare(result_test_0) == false){
    console.log('WARNING!\nTest 1 failed. Sorry, the transform() function is not yet correct.');
}else{
    console.log('Test 1 successful, good job!\n\n---------------\n')
}

// Test for (angle, scale) = (40, 1.01) using NN interpolation 
var solution_test_40 = new Image([[0, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 0]]);
var result_test_40 = transform(test_cross_image, 40, 1.01, 2, 2, 'NN')
console.log('Correct transformation with NN interpolation, (angle, scale) = (40, 1):\n' + solution_test_40.visualize(2)) 
console.log('Your result:\n' + result_test_40.visualize(2));
if(solution_test_40.imageCompare(result_test_40) == false){
    console.log('WARNING!\nTest 2 failed. Sorry, the transform() function is not yet correct.');
}else{
    console.log('Test 2 successful, good job!\n\n---------------\n')
}

Although the sanity check was not applied to real images, we can already see all sorts of undesirable effects. Let's look at the result on a real image. 

## 1.C. Visualizing transformations
[Back to table of contents](#ToC_1_GT_Implementations)

To visualize the effect of the whole operation, we will apply the geometric transformation you just coded to the image `eiffel` . We will do so for the following pairs of angles $\alpha$ and scaling factors $\rho$: ($\alpha$ = 0, $\rho$ = 1), ($\alpha$ = -20, $\rho$=1), ($\alpha$ = 0, $\rho$ = 0.8), and ($\alpha$ = 24, $\rho$ = 2.5), all using the nearest neighbor interpolator.

Run the two next cells, which will apply the transform, convert the results to Python and display them with the viewer.

<div class = 'alert alert-info'>

<b>Note:</b> Did you notice how symmetric was the sanity check from the previous cell? This was necessary, as otherwise, the test image would have been too large. However, it means that certain errors may have not been detected. Use the following cell to check if all the transformations make sense!. 
</div>

In [ ]:
%use javascript
%put output_sequence
// Declare angles and scales
var eiffel_img = new Image(eiffel);
var angles = [0,   0,  24,  -20, -20];
var scales = [1, 0.8, 2.5,   1,    1];
// Compute image centre
var c1 = parseInt(eiffel_img.nx/2);
var c2 = parseInt(eiffel_img.ny/2);
var centers = [[c1, c2], [c1, c2], [c1, c2], [c1, c2], [0, 0]];
// Initialize output list
var output_sequence = [];
// Apply geometric transforms 
for(var i = 0; i < angles.length; i++){
    output_sequence.push(transform(eiffel_img, angles[i], scales[i], centers[i][0], centers[i][1], 'NN').toArray());
}
console.log('Applied ' + angles.length + ' transforms using NN interpolation.')

In [ ]:
%use sos
# Convert serie of image to numpy arrays and declare viewer parameters
image_list = [eiffel] + [np.array(img) for img in output_sequence]
title_list = ['Original image', r"$\alpha=0^{\circ},\rho=1,\mathbf{c} = (220, 324)$", 
              r"$\alpha=0^{\circ},\rho=0.8,\mathbf{c} = (220, 324)$", r"$\alpha=24^{\circ},\rho=2.5,\mathbf{c} = (220, 324)$", 
              r"$\alpha=-20^{\circ},\rho=1,\mathbf{c} = (220, 324)$", r"$\alpha=-20^{\circ},\rho=1,\mathbf{c} = (0, 0)$"]

# Display images
plt.close('all')
eiffel_viewer = viewer(image_list, title=title_list, widgets=True)

## 1.D. Geometric transformations in Python (1 point)
[Back to table of contents](#ToC_1_GT_Implementations)

In Python, geometric transformations can be done easily using OpenCV's [`cv.getRotationMatrix2D`](https://docs.opencv.org/3.4/da/d54/group__imgproc__transform.html#gafbbc470ce83812914a70abfb604f4326) and SciPy's [`scipy.ndimage.affine_transform`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.affine_transform.html#scipy-ndimage-affine-transform) functions. 

`cv.getRotationMatrix2D(center, angle, scale)` takes as parameters
 *  `center`: a tuple containing the center of the transformation $(c_1, c_2)$,
 *  `angle`: the angle of rotation $\alpha$, in degrees, and
 *  `scale`: the scaling factor $\rho$,
 
and returns
 *  `M`: a $2 \times 3$ matrix, where the two first columns contain the rotation matrix, and the last column contains the shifts to be applied after rotating the coordinates $\mathbf{u}$. In case you are wondering, this is because in most cases the expression we showed you [here](#1.A.-Rotation-of-the-coordinate-frame-(1-point)) is implemented as $\mathbf{v} = \mathbf{A}\mathbf{u} + (\mathbf{I}-\mathbf{A}) \mathbf{c}$, which saves a few computations because one does not need to compute $\mathbf{u}-\mathbf{c}$ every time, and $(\mathbf{I}-\mathbf{A}) \mathbf{c}$ is pre-computed and stored in the third column of `M`.

`ndimage.affine_transform(img, M, order=3)` takes as parameters
 *  `img`: the original image,
 *  `M`: the inverse coordinate transformation matrix, mapping output coordinates to input coordinates, in the same $2\times 3$ format used by `cv.getRotationMatrix2D`, and
 *  `order`: the order of the interpolation method to be used (`0` = nearest neighbor, `1` = linear, `3` = cubic, which defaults to `3`),
 
and returns:
 *  `out`: the transformed image.

<div class="alert alert-warning">
    
<b>Technical Note:</b> It turns out that OpenCV and SciPy use different conventions for image locations: while OpenCV uses $(x,y)$ like our JavaScript functions, SciPy uses $(y,x)$, like in the usual NumPy array indexing. Because of the simple structure in rotation matrices, this implies that instead of producing the transformation matrix, inverting it, and then correcting for different indexing conventions, we can directly create the inverse matrix using <code>cv.getRotationMatrix2D</code> and changing <code>center</code> to $(c_2, c_1)$ and <code>scale</code> to $1/\rho$. Figuring why this works out is left as an exercise for those interested, and is the type of practical problem one often faces when interfacing tools from excellent, but different, sources. 
</div>

In the cell below, we use these two functions to create an interactive viewer, where you can play around with the angle and scale parameters. You should understand how to use  `cv.getRotationMatrix2D` and `ndimage.affine_transform` together to perform geometric transformations. For this, look at the function `transform_py()` in the cell below and make sure you understand what it is doing.

The transformation is still performed using the nearest neighbors interpolator. Run the following two cells to play with the transformations.
<div class="alert alert-info">
    
**Note:** To access the sliders, open the widget pannel and click on *Extra widgets*. Then choose a value for $\alpha$ and $\rho$ using the slider. Click on `Apply Transformation` to see the result. If the value of the sliders are hidden, remember that you can access them by scrolling to the right.
</div>

In [ ]:
%use sos
# Define the sliders and button
angle_slider = widgets.FloatSlider(value=0, min=-180, max=180, step=.5, description=r'$\alpha$')
rho_slider = widgets.FloatSlider(value=1, min=0.1, max=4.09, step=0.1, description=r'$\rho$')
c1_slider = widgets.IntSlider(value=eiffel.shape[1]//2, min=0, max=eiffel.shape[1], description='$c_1$')
c2_slider = widgets.IntSlider(value=eiffel.shape[0]//2, min=0, max=eiffel.shape[0], description='$c_2$')
button = widgets.Button(description='Apply transformation')

# Rotate and scale callback function 
def transform_py(image):
    # retreive slider values for angle, scale and center
    angle = angle_slider.value
    scale = rho_slider.value
    c1 = c1_slider.value
    c2 = c2_slider.value
    # Get the rotation matrix and shifts from the angle and the center point
    M = cv.getRotationMatrix2D((c2, c1), angle, 1/scale)
    # Perform the rotation with nearest neighbor (order=0)
    transformed = ndimage.affine_transform(image, M, order=0)
    return transformed

# Visualize angle and scale effect during transformation
plt.close("all")
view = viewer([eiffel], title="Eiffel tower", new_widgets=[angle_slider, rho_slider, c1_slider, c2_slider, button], 
              callbacks=[transform_py], widgets=True)

Finally, let's test your function `transform()` by comparing the result to the Python implementation. In `ndimage.affine_transform()` we will set `order=0` for nearest neighbor interpolation. Make sure that you ran the [first code cell of part 1.C.](#1.C.-Visualizing-transformations) before running this test.

In [ ]:
%use sos
# Get Python transformation
ny, nx = eiffel.shape
M = cv.getRotationMatrix2D((ny//2, nx//2), 24, 1/2.5)
transformed = ndimage.affine_transform(eiffel, M, order=0)
# Check that the two images are identical
try:
    np.testing.assert_array_almost_equal(transformed, output_sequence[2])
    print('Congratulations! Your function gives the same result as some of the most accepted image processing libraries in the world.')
except:
    print('WARNING!\nYour transform function does not produce the same output as OpenCV and SciPy ndimage.\nSomething is not yet correct' +
          ' with your code.\nInspect closely the following images to look for differences.')
    plt.close('all')
    view = viewer([transformed, np.array(output_sequence[2])], title=["SciPy's output (Ground truth)", 'Your JS output'], widgets=True)

### Multiple Choice Question
Use the interactive viewer two cells above to answer the following question **for 1 point**. 

* What parameters should be used to align the eiffel tower vertically, with the main axis of the tower in the middle of the image while only keeping the upper part (only the region between the second and third platform) of the tower?


<div class=" alert alert-info">

<b>Note:</b> To answer, change the variables <code>alpha</code>, <code>rho</code>, <code>c_1</code> and <code>c_2</code> in the cell below to the values corresponding to your choice. Then run the 4 cells below to check that your answers is valid. 
</div>

In [ ]:
%use sos
# Modify the variable answer
alpha = None
rho = None
c_1 = None
c_2 = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not (alpha >= -90 and alpha <= 90):
    print('WARNING!\nChoose a suitable range for alpha.')

In [ ]:
%use sos
# Sanity check
if not (rho >= 0 and rho <= 4):
    print('WARNING!\nChoose a suitable range for rho.')

In [ ]:
%use sos
# Sanity check
if not (c_1 >= 0 and c_1 <= 440):
    print('WARNING!\nChoose a suitable range for c_1.')

In [ ]:
%use sos
# Sanity check
if not (c_2 >= 0 and c_2 <= 690):
    print('WARNING!\nChoose a suitable range for c_1.')

# 2. B-spline interpolation scheme and linear interpolation
[Back to table of contents](#ToC_1_GT_Implementations)

Now that you know how to transform images by an angle $\alpha$ and a scaling factor $\rho$, it is time to think about the quality of the result. As you saw in the first section, nearest neighbor interpolation already gives good results, but can we do better?

In this section we will deal with linear interpolation. First you will implement a linear interpolation in JavaScript and then we will once again compare it to the SciPy version, which uses linear interpolation by default.

## 2.A. Understanding the method (1 point) 
[Back to table of contents](#ToC_1_GT_Implementations)

To improve the quality of the transformed image, we propose to compute the output image $g(\mathbf{x})$, where $\mathbf{x} = (x,y)$, as a weighted sum of shifted two-dimensional B-spline basis functions $\boldsymbol{\beta}^n(\mathbf{x}) = \beta^n(x) \beta^n(y)$, i.e.,

$$
    g(\mathbf{x})=\sum_{\mathbf{k}\in\mathbb{Z}^2}c[\mathbf{k}]\boldsymbol{\beta}^n(\mathbf{x}-\mathbf{k})\mbox{, where } \mathbf{k}=(k_1,k_2)
$$

is the vector of image indices. 
<div class="alert alert-info">

<b>Note:</b> If you don't remember everything you need to remember about B-splines, revise Chapter 7.2 of your course notes.
</div>

In this section, we propose to implement a *linear interpolation* $(n = 1)$, while in the next ([Section 3](#3.-Cubic-interpolation)), we will propose to implement a *cubic interpolation*  $(n = 3)$. Note that, as you saw in the course, the basis functions $\beta^1(x)$ are interpolant, meaning that the coefficients $c[k]$ will simply be the samples of the input image, i.e., $c[k] = s[k]$. However, the basis functions $\beta^3(x)$ are not interpolant, and a pre-filter is required to compute the coefficients $c[k]$ from the input image $s[k]$.

**For 1 point**, answer the following MCQ:
 * Which are the values at the origin $\beta^1(0)$ and $\beta^3(0)$?, which is the size of their support (the set $\lbrace x\in\mathbb{R} : \beta^n(x)>0 \rbrace$)?

1. $\beta^1(0) = \frac{2}{3}$ and its support is of size $4$, and $\beta^3(0) = 1$ and its support is of size $2$,
2. $\beta^1(0) = 1$ and its support is of size $2$, and $\beta^3(0) = 1$ and its support is of size $2$,
3. $\beta^1(0) = \frac{2}{3}$ and its support is of size $4$, and $\beta^3(0) = \frac{6}{8}$ and its support is of size $3$,
4. $\beta^1(0) = 1$ and its support is of size $2$, and $\beta^3(0) = \frac{2}{3}$ and its support is of size $4$.

<div class = 'alert alert-info'>

<b>Note:</b> To answer the question, change the value of the variable <code>answer</code> in the next cell. The cell below it is for you to check that your answer is valid. If you have any doubts, go to your course notes.
</div>

In [ ]:
%use sos
# Modify the variable answer
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in [1, 2, 3, 4]:
    print('WARNING!\nChoose one of 1, 2, 3 or 4.')

## 2.B. Linear interpolation (3 points)
[Back to table of contents](#ToC_1_GT_Implementations)

The image below illustrates how two-dimensional linear interpolation works. In the example, we only have $4$ pixels and we're interpolating the value of the image at the location $(dx,dy)$. In practice, all examples can be reduced to this, since the $4$ neighboring pixels are the only ones that are going to have an effect on the interpolation, and $(dx,dy)$ can be seen as the distances to the integers immediately below the locations $x$ and $y$ of interest.

The general procedure is as follows. First, the $2\times 2$ neighborhood of the image around the wanted location is extracted. For simplicity, we will refer to them as $s[k,l]$, with $k,l=0$ corresponding to the integers immediately below $(x,y)$, and $k,l=1$ corresponding to the integers immediately above $(x,y)$. Then, the $4$ weights to apply to each of them are extracted from the values of $\beta^1(\mathbf{x}-(0,0))$, $\beta^1(\mathbf{x}-(0,1))$, $\beta^1(\mathbf{x}-(1,0))$, and $\beta^1(\mathbf{x}-(1,1))$ at the location $\mathbf{x}=(dx,dy)$.  Finally, because in linear interpolation $c[k]=s[k]$, the interpolation is the linear combination of the four pixels with these weights. 

<img src="images/bilinear_interpolation_showcase.png" alt="Drawing" style="width: 400px;"/>

From $dx$ and $dy$, the weights given by $\beta^1(\mathbf{x})$ are, for $k,l\in\lbrace 0,1\rbrace$,

$$
\beta^1[k, l]=
    \begin{bmatrix}
        (1-dx)(1-dy) & dx(1-dy) \\
        (1-dx)dy & dxdy
    \end{bmatrix}
$$

and the interpolated value becomes

$$
    v = \sum_{k=0}^{1}\sum_{l=0}^{1}\beta^1[k,l]s[k,l]\,,
$$

with the convention for $s[k,l]$ explained above.

As the first step, **for 1 point**, implement the function `getLinearSpline(dx, dy)` that returns a $2\times 2$ `Image` object that represents the weight $\beta^1[k, l]$ for $(dx, dy)\in[0,1]^2$ in the cell below. For further information, you can look at slides 7-11 amd 7-12 of your [course notes](https://moodle.epfl.ch/pluginfile.php/2749644/mod_resource/content/0/IP-Chap%207_handout.pdf).

In [ ]:
%use javascript

function getLinearSpline(dx, dy){
    // Check for correct input parameters
    if(dx < 0 || dx > 1 || dy < 0 || dy > 1){
        throw new Error("Argument for linear B-spline is outside of the expected range [0, 1].");       
    }
    // Create the 2x2 Image object
    var beta = new Image(2, 2);
    
    // YOUR CODE HERE
    
    return beta;
}

Before proceeding, we will perform a sanity check as usual. Verify that the output values also make sense to you.

In [ ]:
%use javascript
// Check some simple interpolation cases
var in_values = [[0, 0], [1, 0], [0.5, 0.5], [0.75, 0.25]];
var out_values = [[[1, 0], [0, 0]], [[0, 1], [0, 0]], [[0.25, 0.25], [0.25, 0.25]], [[0.188, 0.563], [0.063, 0.188]]];
var check_error = false;
for(var i = 0; i < in_values.length; i++){
    console.log('Linear spline weights for dx=' + in_values[i][0] + ', dy=' + in_values[i][1] + ':')
    test_img = getLinearSpline(in_values[i][0], in_values[i][1])
    console.log(test_img.visualize())
    if(test_img.imageCompare(new Image(out_values[i]), tol=1e-3) == false){
        console.log('WARNING!\nThe result for dx=' + in_values[i][0] + ', dy=' + in_values[i][1] + ' is not correct.\n' +
                    'Expected output:\n' + (new Image(out_values[i])).visualize() + '---------------\n');
        check_error = true;
    }else{
        console.log('This is correct.\n---------------\n');
    }
}
if(!check_error){
    console.log('Nice, the B-splines seem to be correct.')
}

In the next cell, **for 2 points**, implement the function `linear_interpolator()`, which returns the linearly interpolated value at the position $(x,y)$.

The function `linear_interpolator(img, x, y)` takes as parameters
 * `img`: the image to interpolate the value from,
 * `x`: the x-coordinate at which we want to obtain an interpolated value,
 * `y`: the y-coordinate at which we want to obtain an interpolated value.
 
The function outputs the linearly interpolated value in the variable `out`.

<div class="alert alert-info">

**Hints:**
* Use the function `getLinearSpline()` you implemented above to obtain the right weights for each of the surrounding pixels.
* If you use `img.getNbh(x, y, w, h)` with an **even** width $w$ and height $h$, the returned neighborhood is from $x-\frac{w}{2}$ to $x+\frac{w}{2}-1$ and from $y-\frac{h}{2}$ to $y+\frac{h}{2}-1$. Make sure that you get the correct pixels for your interpolation!
* To get the floor $\lfloor x \rfloor$ of a number $x$ in JavaScript, you can use the function `parseInt()`.
</div>

In [ ]:
%use javascript

function linear_interpolator(img, x, y){
    var out = 0;
    
    // YOUR CODE HERE
    
    return out 
}

Now run the next cell to perform a sanity check on this function as well.

In [ ]:
%use javascript 
// Let's check the linear interpolator 
var test_linear = new Image([[1,2,3,4,5], [6,7,8,9,10], [11,12,13,14,15]]);
var locs = [[3, 1], [1.5, 1.5]];
var vals = [9, 10];
var check_error = false;
console.log('Input image for sanity check:\n' + test_linear.visualize())
for(var i = 0; i < locs.length; i++){
    var test_val = linear_interpolator(test_linear, locs[i][0], locs[i][1]);
    console.log('Your interpolated value at location (x=' + locs[i][0] + ', y=' + locs[i][1] + ') is ' + test_val + '.');
    if(test_val != vals[i]){
        console.log('WARNING!\nThe linear_interpolator is not yet correct. The expected value at location (x=' + 
                    locs[i][0] + ', y=' + locs[i][1] + ') is ' + vals[i] + '.\n---------------\n');
        check_error = true;
    }else{
        console.log('This is correct.\n---------------\n')
    }
}
if(!check_error){
    console.log('Nice, the linear_interpolator function passed the sanity check!');
}

In the next two cells, we will compare the output of your `transform` function to the output of `ndimage.affine_tranform` when both are using linear interpolation. Run the two cells to see if your transform function works correctly with linear interpolation.

<div class = 'alert alert-info'><b>Note</b>: After running the next two cells, you will see either see a winning message, or you transformed image display with some regions in red. These regions show the differences between your implementation and the Python implementation. Use this info to debug your linear interpolation.
</div>

In [ ]:
%use javascript
%put transformed_js
// JavaScript transformation 
// Remember that we defined transform(img, angle, scaling, c1, c2, interpolator)
var transformed_js = transform(new Image(eiffel), -72, 1.3, 282, 355, interpolator='LINEAR').toArray();

In [ ]:
%use sos
# Python transformation (order=1 corresponds to linear interpolation)
M = cv.getRotationMatrix2D(center=(355, 282), angle=-72, scale=1/1.3)
transformed = ndimage.affine_transform(eiffel, M, order=1)
# Check that the images are identical
if np.allclose(transformed, np.array(transformed_js)):
    print('Good job! The trasformation with linear interpolation gives the same result as OpenCV.')
else:
    print('WARNING!\nThe transformation with linear interpolation is not correct.\nSee the images below and try to find the source of the error.')
    plt.close('all')
    title_list = ['Python transformation', 'Your transformation']
    view = viewer([transformed, np.array(transformed_js)], title=title_list, widgets=True, subplots=(1,2), compare=True)


# 3. Cubic interpolation 
[Back to table of contents](#ToC_1_GT_Implementations)

As you have seen in the previous section, the linear interpolation using linear splines already provides quite good results, but let's try to do better with a cubic interpolation, i.e., using cubic splines. Unlike with linear B-splines, where the coefficients are simply the values of the input image, for the cubic interpolator we will need to implement a prefilter to compute the coefficients $c[\mathbf{k}]$.

## 3.A. Prefilter (3 points)
[Back to table of contents](#ToC_1_GT_Implementations)

Luckily, the computation of the values $c[\mathbf{k}]$ from the image $s[\mathbf{k}]$ is separable. A fast implementation of the one-dimensional prefilter to obtain the coefficients $c[k]$ in the cubic interpolator is obtained by a cascade of recursive filters. This cascade is schematized below, where $c_{0}=6$ and ${a}=\sqrt{3}-2$:

<img src="images/diagram.jpg" alt="Drawing" style="width: 500px;"/>

Here, your job will be to implement this one-dimensional filter in the function `recursiveExponentialFilter()` below. 

Before that, however, run the cell below to define the function `cubicSplineCoefficients()`, which calls `recursiveExponentialFilter()` first over the rows and then over the columns of an image to compute the coefficients $c[\mathbf{k}]$ of an image $s[\mathbf{k}]$.

There, we also define the functions `initialValueCausal(signal, a)` and `initialValueAntiCausal(signal, a)`, which will help you in your implementation and take as input parameters
* `signal`: a one-dimensional `Image` object representing a row/column of an image, and
* `a`: the value of $a$ in the diagram above,

and return

* `out`: the initial value for the causal and anticausal filters, respectively.

In [ ]:
%use javascript
// Function that returns the 2D cubic spline coefficients by applying the separable pre-filter
function cubicSplineCoefficients(img){
    var out = new Image(img.shape()); 
    // Iterate over rows and apply 1D prefilter
    for(var y = 0; y < img.ny; y++){
        var row = img.getRow(y);
        var prow = recursiveExponentialFilter(row);
        out.putRow(y, prow); 
    }
    // Iterate over columns and apply 1D prefilter on the result
    for(var x = 0; x < img.nx; x++){
        var column = out.getColumn(x);
        var pcolumn = recursiveExponentialFilter(column);
        out.putColumn(x, pcolumn); 
    }
    return out;
}

// Provided methods that return the appropriate initial values
function initialValueCausal(signal, a){
    // Set the precision of the initial value
    var k0 = Math.min(12, signal.nx); 
    var polek = a;
    // Approximate the initial value with the pixels following the boundary
    var v = signal.getPixel(0, 0); 
    for(k = 1; k < k0; k++){
        v += polek * signal.getPixel(k, 0);
        polek *= a;
    }
    return v; 
}

function initialValueAntiCausal(signal, a){
    var n =  signal.nx;
    return (a / (a * a - 1.0)) * (signal.getPixel(n-1, 0) + a * signal.getPixel(n-2, 0)); 
}

In the next cell, **for 3 points**, implement the function `recursiveExponentialFilter(s)` that performs the one-dimensional recursive filtering on a single row/column. This function has as input parameter:

* `s`: a one-dimensional `Image` object representing a single row/column $s[k]$. `s` is of size $1 \times w$, i.e., it is of width $w$ and height $1$,

and returns:

* `c`: a one-dimensional `Image` object with the same shape that contains the coefficients $c[k]$ corresponding to the input signal $s[k]$. 

<div class="alert alert-info">
    
**Hints:** 
* $\operatorname{H}(z) = \sum_{m=0}^{M-1}b_m z^{-m} \left/ \sum_{n=0}^{N-1}a_n z^{-n}\right.$ is equivalent to $\sum_{n=0}^{N-1}a_n y[k-n] = \sum_{m=0}^{M-1}b_m x[k-m]$.
* Although exponential filters have an infinite impuse responses (IIR), every value you set when applying the filter contains all the information about the previous (or future) values of the original signal. In practice, then, at each step you only need the last value you set and the value of the input signal at the current position. Revise your course notes for more information. 
* Use the function `initialValueCausal` to approximate the value of $y[0]$ for the causal filter and `initialValueAntiCausal` to approximate the value of $y[w-1]$ for the anticausal filter.
</div>

<div class="alert alert-warning">
    
**Note:** Remember that the method `Image.setPixel()` (and `Image.getPixel()`) **use the convention $(x,y)$ for pixel positions**. Because you are using one-dimensional `Image` objects with a height of 1 element, the second parameter (the $y$ position of the pixel being accessed) should always be 0.
</div>

In [ ]:
%use javascript
// Function that computes the cubic coefficients using a recursive exponential filter
function recursiveExponentialFilter(s){
    // Initialize the coefficients to contain the same values as the input signal
    var c = s.copy();
    // Extract the signal length (width)
    var n = s.nx;
    
    // YOUR CODE HERE
    
    return c;
}

As a sanity check, we will apply your function to an impulse sequence of length 11. Run the next cell to do so.

In [ ]:
%use javascript
// Define test image and expected output
var impulse = new Image([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]);
var solution = new Image ([[-0.006, 0.012, -0.042, 0.158, -0.588, 2.196, -0.588, 0.158, -0.042, 0.012, -0.006]]);
// Get the coefficients
var test = cubicSplineCoefficients(impulse);
console.log('Input sequence:\n' + impulse.visualize())
console.log('Your cubic spline coefficients:\n' + test.visualize())
// Compare the output to the solution
if(test.imageCompare(solution, tol=1e-3) == false){
        console.log("WARNING!\nThe recursive exponential filter is not working properly, the expected cubic spline coefficients are:\n" + 
                    solution.visualize());
}
else{
    console.log('Congratulations! Your implementation passed the sanity check.' +  
                '\nHowever, this is not a guarantee that it works well for any input!')
}

## 3.B. Cubic interpolation (2 points) 

[Back to table of contents](#ToC_1_GT_Implementations)

Now that we have the correct coefficients, we can complete the `cubic_interpolator()` function below. 

To simplify your task, however, we provide the function `getCubicSpline(dx, dy)` that returns a $4\times4$ `Image` object that represents the values of the two-dimensional cubic B-spline $\beta^3[k, l]$ for some values $dx,dy\in[0,1]^2$ defined as in [Section 2.B.](#2.B.-Linear-interpolation-(3-points)), and corresponds to your `getLinearSpline()` function but for the case of cubic interpolation. Run the next cell to define this function.

In [ ]:
%use javascript
// Function that returns the cubic spline coefficiants as a 4x4 Image object
function getCubicSpline(dx, dy){
    var out = new Image(4, 4); 
    // Check input parameters
    if(dx < 0 || dx > 1 || dy < 0 || dy > 1){
        throw new Error("Argument for cubic B-spline is outside of the expected range [0, 1].");       
    }
    // Function that calculates the 1D cubic B-spline values
    function getV(t){
        var v = new Array(4);
        v[0] = (1.0-t)**3 / 6.0;
        v[1] = (2.0 / 3.0) - 0.5 * t**2 * (2 - t);
        v[3] = t**3 / 6.0;
        v[2] = 1.0 - v[3] - v[1] - v[0];
        return v
    }
    var vx = getV(dx);
    var vy = getV(dy);
    // Calculate the 2D cubic spline coefficients
    for(var k = 0; k < 4; k++){
        for(var l = 0; l < 4; l++){
            out.setPixel(k, l, vx[k] * vy[l]);
        }
    }
    return out;
}

**For 2 points** implement the method `cubic_interpolator(c, x, y)` in the cell below. This function takes as input parameters

 * `c`: an `Image` object containing the coefficients $c[\mathbf{k}]$ calculated by the function `cubicSplineCoefficients()`,
 * `x`: the x-coordinate at which we want to obtain an interpolated value,
 * `y`: the y-coordinate at which we want to obtain an interpolated value,

and returns

* `out`: the interpolated value.

<div class="alert alert-info">

<b>Hints:</b><ul>
<li>Use the function <code>getCubicSpline()</code> provided for you to obtain the cubic spline of size $4 \times 4$ for each interpolation location.</li>
<li>If you need further help, check the advice we gave you just before you implemented <code>linear_interpolator()</code>.</li></ul>
</div>

In [ ]:
%use javascript

function cubic_interpolator(c, x, y){
    var out = 0;
    
    // YOUR CODE HERE
    
    return out;
}

Now run the next cell perform a quick sanity check as always.

In [ ]:
var test_cubic = new Image([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15]]);
console.log('Input image for sanity check:\n' + test_cubic.visualize());
console.log('Your interpolated value at location (x=3, y=1) is ' + cubic_interpolator(test_cubic, 3, 1) + '.');
var check_error = false;
if(cubic_interpolator(test_cubic, 3, 1) != 9){
    console.log('WARNING!\nThe linear_interpolator is not yet correct. The expected value at location (x=3, y=1) is 9.\n---------------\n');
    check_error = true;
}else{
    console.log('This is correct.\n---------------\n')
}
console.log('Your interpolated value at location (x=1.5, y=1.5) is ' + cubic_interpolator(test_cubic, 1.5, 1.5).toPrecision(4) + '.');
if(Math.abs(cubic_interpolator(test_cubic, 1.5, 1.5) - 9.896) > 0.001){
    console.log('WARNING!\nThe linear_interpolator is not yet correct. The expected value at location (x=1.5, y=1.5) is 9.896.\n---------------\n');
    check_error = true;
}else{
    console.log('This is correct.\n---------------\n')
}
if(!check_error){
    console.log('Nice, the cubic_interpolator function passed the sanity check!');
}

Once again, run the next two cells to compare your implementation with the one from Python again.

<div class = 'alert alert-info'><b>Note</b>: After running the next two cells, you will see either see a winning message, or you transformed image display with some regions in red. These regions show the differences between your implementation and the Python implementation. Use this info to debug your cubic interpolation.
</div>

In [ ]:
%use javascript
%put transformed_js
// JavaScript transformation
var transformed_js = transform(new Image(eiffel), 45, 0.8, 150, 250, 'CUBIC').toArray();

In [ ]:
%use sos
# Python transformation
M = cv.getRotationMatrix2D((250, 150), 45, 1/0.8)
transformed = ndimage.affine_transform(eiffel, M, order=3)
# Check that the difference is less than 0.1% (boundary issues caused by different implementations of the exponential filters)
if np.count_nonzero(transformed.astype(np.int) - np.array(transformed_js, dtype=np.int))/np.size(transformed) < 1e-3:
    print('Good job! The trasformation with cubic interpolation is correct.')
else:
    print('WARNING!\nThe transformation with cubic interpolation is not yet correct.\nCheck the images to try to find the source of the error.')
    plt.close('all')
    viewer([transformed, np.array(transformed_js)], title=['SciPy (Ground truth)', 'JS'], widgets=True, compare=True)

Now, we are sorry about the difficulty of computing the coefficients $c[\mathbf{k}]$ and then applying cubic interpolation. You should not forget about this step! As Michaël Unser wrote when presenting his work to reinsure the importance of the pre-filter and the applications of the cubic spline:

<table><tr>
<td> 
  <p align="center" style="padding: 50px">
    <em style="font-weight: bold">It is worth mentioning that many authors in image processing leave out the essential prefitering step [...]. This has a catastrophic effect on performance and perpetuates the incorrect belief that high-order B-spline interpolation results in increased image blurring.</em>
    <br> <br>
    <em style="color: grey;padding-left: 100px">M. Unser, "Splines: a perfect fit for signal and image processing," in IEEE Signal Processing Magazine, vol. 16, no. 6, pp. 22-38, Nov. 1999, DOI: <a href="https://ieeexplore.ieee.org/abstract/document/799930">10.1109/79.799930</a></em>.
  </p> 
</td>
</tr></table>
 
As a quick demonstration, run the next cell where we will plot a transformation with and without the prefilter. Zoom into the high frequency regions to see the blurring caused by lack of knowledge!

In [ ]:
%use sos
# Get transformations
M = cv.getRotationMatrix2D((590, 100), 35, 1/1.5)
transformed_prefilt = ndimage.affine_transform(eiffel, M, order=3)
transformed_no_prefilt = ndimage.affine_transform(eiffel, M, order=3, prefilter = False)

# Visualize
plt.close('all')
view = viewer([transformed_prefilt, transformed_no_prefilt], title=['Prefiltered (Ground truth)', 'Not prefiltered (Wrong)'], 
              widgets=True, subplots=(1,2), joint_zoom=True)

## 3.C. Visual comparison between the three interpolators
[Back to table of contents](#ToC_1_GT_Implementations)

Finally, let us have a quick look at the difference between the three types of interpolators. Run the two cells below to apply the same transformation with the three different interpolators you implemented in this lab and visualize them. This is mostly for you to see the value in what you have been doing in this last section. In the second part of this lab, we will also compare the three interpolation methods numerically in order to see what we gain from a more complex interpolation.
<div class='alert alert-info'>

**Note:** Feel free to change the parameters of the transformation to see different results.
</div>

In [ ]:
%use javascript
%put transformed_nn transformed_lin transformed_cub
// Define the transformation parameters
var angle = 30;
var scaling = 4.2;
var cx = 245;
var cy = 360;

// Perform transformation with the three different interpolators
var eiffel_img = new Image(eiffel);
var transformed_nn = transform(eiffel_img, angle, scaling, cx, cy, interpolator='NN').toArray();
var transformed_lin = transform(eiffel_img, angle, scaling, cx, cy, interpolator='LINEAR').toArray();
var transformed_cub = transform(eiffel_img, angle, scaling, cx, cy, interpolator='CUBIC').toArray();

In [ ]:
%use sos
# Visualize the results
plt.close('all')
view = viewer([np.array(transformed_nn), np.array(transformed_lin), np.array(transformed_cub)], 
              title=['Nearest neighbor', 'Linear', 'Cubic'], subplots=(2,2), joint_zoom=True, widgets=True)

<div class="alert alert-success">
    
<p><b>Congratulations on finishing the first part of the geometric transformation lab!</b></p>
<p>
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to <a href="https://moodle.epfl.ch/mod/assign/view.php?id=1146081">Moodle</a>, in a zip file with other notebook of this lab.
</p>
</div>

* Keep the name of the notebook as: *1_GT_Implementations.ipynb*,
* Name the zip file: *Geometric_Transformation_lab.zip*.